# INFO 3350/6350

## Lecture 02: Dictionary methods

## To do

* Readings
    * "Emotional Arcs ..." and others for today
    * Two articles and one project to read/review for Wednesday
* **Response posts.** See [Canvas for schedule and instructions](https://canvas.cornell.edu/courses/57246/discussion_topics/618217). 
    * The readings themselves are *always* required. Read every week. Respond only as scheduled.
    * This week: NetIDs `a*-g*`
* PS0 due Thursday night at 11:59pm
    * If you're still on the waitlist, you can hold off on PS0. We'll accept it without penalty after you're admited.
* Speaking of the waitlist ...
    * Keep attending lecture if you want to be admitted. I'll have an update on Wednesday.
* Remember that Ed is the place for questions about any aspect of the course. 
    * Follow up in office hours as needed.
    
## What are dictionary methods and what do they have to do with word counts?

Dictionary methods use a *dictionary* or *lexicon* to measure textual properties on the basis of known values associated with words or phrases. You can think of them as a pre-determined mapping between words and features.

### Example: How much "animalness" does a text contain?

The [text](https://helenadailyenglish.com/short-stories-in-english-a-cat-and-a-dog.html):

> The black kitten jumped up onto the chair. It looked down at the white puppy. The dog was chewing on a bone. The cat jumped onto the dog. The dog kept chewing the bone. The cat played with the dog’s tail. The dog kept chewing the bone.

In [ ]:
from   collections import Counter
from   nltk.tokenize import word_tokenize
import string

story = '''The black kitten jumped up onto the chair. 
It looked down at the white puppy. 
The dog was chewing on a bone. 
The cat jumped onto the dog. 
The dog kept chewing the bone. 
The cat played with the dog's tail. 
The dog kept chewing the bone.
A bird looked on.
'''

# case fold and tokenize, then remove punctuation
story_tokens = Counter(word_tokenize(story.lower()))
for i in string.punctuation:
    del story_tokens[i]

# word count
story_token_count = sum(story_tokens.values())
print("Story tokens:", story_token_count)

# count cat words
cat_token_set = set(['cat', 'kitty', 'kitten', 'feline'])
cat_token_count = sum([story_tokens[i] for i in cat_token_set])
print("Cat tokens:", cat_token_count)

# Calculate "animalness" = cat tokens / all tokens
print(f"'Animalness': {100* cat_token_count / story_token_count:.1f}%")

In [ ]:
# a more comprehensive list of animals ...
animal_token_set = set(['cat', 'kitty', 'kitten', 'feline', 'dog', 'puppy', 'bird', 'chick'])

animal_token_count = sum([story_tokens[i] for i in animal_token_set])
print("Animal tokens:", animal_token_count)

# Calculate "animalness"
print(f"'Animalness': {100* animal_token_count / story_token_count:.1f}%")

#### Problems with this method?

Take 10 seconds to consider limitations or problems with this method of measuring the "animalness" of a story. Then put your thoughts in the chat.

How might one try to address these issues?

## A common use case: sentiment and emotion

We want to determine the *sentiment* of a text and of the individual sentences from which the text is composed.

"Sentiment" can mean a lot of things:

* Positive and negative feelings
* Emotional intensity (could be good or bad)
* Amount or intensity of other emotions (joy, surprise, awe, fear, etc.)
* Maybe even "sentimentalness" (roughly, "nostalgia")

Today, we'll focus on sentiment as the expression of positive and negative feelings at the token level. This is a common case in many text anlaysis problems, though it has enough problems that you probably shouldn't use it for any sort of general-purpose problem.

## Supervised and unsupervised

There are two broad ways we could approach the task of senitment analysis:

* **Unsupervised** methods start with **known-informative features** and produce labels or scores from those features.
* **Supervised** methods start with **labeled data** and try to learn the features that best predict the labels.
* Advantages and disadvantages of each
  * In short: upfront labeling costs vs. later validation costs
  
## Warning

Every semester, we see a surprising number of student exams/projects that are built around unsupervised sentiment analysis. These projects tend to be weak, since they rely on a method covered in lecture 2 that has lots of known limitations and many superior alternatives. Do not fall into this trap.

## Our method

We will work, for now, with **unsupervised** sentiment analysis. But there are lots of supervised approaches, too.

Specifically, we're going to use lexical (dictionary-based) methods that assign one or more emotions to a subset of English words. We will assume that each of those words in a given text is an indication that the text contains the corresponding emotion. We can them sum up the emotions over all words in the text to get a measurement of that text's net emotional content.

**Quick exercise:** Do you expect this method to work? Do you think there are cases where it might work especially well or poorly? What are some potential problems?

## Example case

* From Jockers' [*Syuzhet* vignette](https://cran.r-project.org/web/packages/syuzhet/vignettes/syuzhet-vignette.html). 
 * Note that this is an R package. We can't use it directly.
* "Syuzhet" = "plot" or "subject" in Russian; it refers to narrative order, rather than to the "true" order of the narrative's underlying events (which is called the *fabula*). See [Russian Formalism](https://en.wikipedia.org/wiki/Russian_formalism).
* To see the difference, think about a flashback that occurs near the end of a story.

Consider the following story (or "story"):

> I begin this story with a neutral statement.  
  Basically this is a very silly test.  
  You are testing the Syuzhet package using short, inane sentences.  
  I am actually very happy today.  
  I have finally finished writing this package.  
  Tomorrow I will be very sad.  
  I won't have anything left to do.  
  I might get angry and decide to do something horrible.  
  I might destroy the entire package and start from scratch.  
  Then again, I might find it satisfying to have completed my first R package.  
  Honestly this use of the Fourier transformation is really quite elegant.  
  You might even say it's beautiful!  

### Score some sentences ...

By show of hands, how many think each sentence is:
* Positive
* Negative

The sentences:
1. **Basically this is a very silly test.**
1. **You are testing the Syuzhet package using short, inane sentences.**  
1. **I have finally finished writing this package.**  
1. **I won't have anything left to do.** 
1. **Honestly this use of the Fourier transformation is really quite elegant.**  

In [ ]:
# Enter eyeballed values from in-class survey ...
# Range [-2, 2] strong negative to strong positive
silly    = 0.
inane    = 0.
finished = 0.
left     = -0.75
fourier  = 1.7

# scores for all sentences, includings ones not recorded above
human_scores = [
    0,
    silly,
    inane,
    2,
    finished,
    -2,
    left,
    -2,
    -2,
    1,
    fourier,
    1.5
]

print("Human scores by sentence:", human_scores)
print("Overall human sentiment score:", round(sum(human_scores),3))

**Confidence check:** Does this strike us as a reasonable summary of the overall positive-negative affect of the sample story? If not, why not?

### Ingest and tokenize the example "story"

Notice that our output data structure is a list of lists. The "outer" list contains sentences. The "inner" lists each contain the tokens in one sentence.

In [ ]:
from nltk import sent_tokenize, word_tokenize

# The story. Why triple quotes?
story = '''\
  I begin this story with a neutral statement.  
  Basically this is a very silly test.  
  You are testing the Syuzhet package using short, inane sentences.  
  I am actually very happy today. 
  I have finally finished writing this package.  
  Tomorrow I will be very sad. 
  I won't have anything left to do. 
  I might get angry and decide to do something horrible.  
  I might destroy the entire package and start from scratch.  
  Then again, I might find it satisfying to have completed my first R package. 
  Honestly this use of the Fourier transformation is really quite elegant.  
  You might even say it's beautiful!'''

# Tokenize the story
tokens = [word_tokenize(sent.lower()) for sent in sent_tokenize(story)]
print("Sentences:", len(tokens))
print("Total tokens:", sum([len(sent) for sent in tokens]))
print("\nSample sentences:", tokens[0:2])

### Set up sentiment dictionaries

We want to compare a couple of them. Specifically, we'll use:

* NLTK's copy of Hu and Liu's lexicon ([paper](https://www.cs.uic.edu/~liub/publications/kdd04-revSummary.pdf) | [dataset](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html))
* Mohammad's [EmoLex](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) from the Canadian National Research Council (NRC). 
    * Crowd-sourced word associations.
    * Some are ... questionable?
      * "fall" -> `SADNESS`; "pregnancy" -> `DISGUST|NEGATIVE`

Are these good and suitable choices for our task? Let's (begin to) find out!

In [ ]:
from   collections import defaultdict
import nltk
import os
import random

nltk.download('opinion_lexicon') # Need to download this the first time used

# NLTK simple lexicon (from Hu and Liu (2004))
nltk_lexicon = {
    'positive' : set(nltk.corpus.opinion_lexicon.positive()), # Why cast to a set?
    'negative' : set(nltk.corpus.opinion_lexicon.negative())
}

# Print a sample of the NLTK lexicon
print('NLTK lexicon sample')
for key in nltk_lexicon.keys():
    print(f'{key}:', random.sample(tuple(nltk_lexicon[key]), 5))
    
# NRC EmoLex lexicon (from Mohammad, http://sentiment.nrc.ca/lexicons-for-research/)
# No package for this, just read the data from a local file
emolex_file = os.path.join('..', 'data', 'lexicons', 'emolex.txt')
nrc_lexicon = defaultdict(dict) # Like Counter(), defaultdict eases dictionary creation
with open(emolex_file, 'r') as f:
    # emolex file format is: word emotion value
    for line in f:
        word, emotion, value = line.strip().split()
        nrc_lexicon[word][emotion] = int(value)
        
# Print a sample of the NRC EmoLex lexicon
print('\nNRC lexicon sample')       
for key in random.sample(tuple(nrc_lexicon.keys()), 2):
    print(f'{key}:', nrc_lexicon[key])

### Define scoring function

Set up a function to score a word as positive or negative using either the NLTK or NRC dictionary.

In [ ]:
def word_sentiment_score(word, method='nrc', lex=nrc_lexicon):
    '''
    Takes a word, optional method in ['nrc', 'nltk'], and optional lexicon dictionary.
    Returns 1 (if positive), -1 (if negative), 0 (neutral), or None (not in lex).
    '''
    word = word.lower() # Handle non-case-folded inputs
    if method.lower() == 'nrc':
        if word in lex: # Only score words that are in the lexicon
            pos = lex[word]['positive']
            neg = lex[word]['negative']
            if pos == neg: # Ties (mostly 0==0) return zero
                return 0
            elif pos > neg:
                return 1
            else:
                return -1
    elif method.lower() == 'nltk':
        if word in lex['positive']:
            return 1
        elif word in lex['negative']:
            return -1
    else:
        raise NameError("Method not in ['nrc', 'nltk']")
    return None # If word not in lexicon, return None (not zero). Why do this?

In [ ]:
print("NRC  'analyst':", word_sentiment_score('analyst'))
print("NLTK 'analyst':", word_sentiment_score('analyst', method='nltk', lex=nltk_lexicon))

### Score our example sentences

For reference, here are our class-crowd-sourced scores:

In [ ]:
# Human scores
print("Method: human")
print("Sentence scores:", human_scores)
print("Summary score:", sum(human_scores))

In [ ]:
# Calculate and compare dictionary scores

# Scoring methods for input
methods = {
    'nltk':nltk_lexicon,
    'nrc' :nrc_lexicon
}

# Calculate, record, and print scores
method_scores = {} # To store results
for method in methods:
    sentence_scores = [] # Could rewrite next few lines as a list comprehension
    for sent in tokens:
        sentence_score = 0
        for word in sent:
            word_score = word_sentiment_score(word, method=method, lex=methods[method])
            if word_score != None:
                sentence_score += word_score
        sentence_scores.append(sentence_score)
    method_scores[method] = sentence_scores # Save sentence-level scores
    # Print results
    print("Method:", method)
    print("Sentence scores:", sentence_scores)
    print("Summary score:", sum(sentence_scores),'\n')

In [ ]:
# Visualize and print the story for discussion
import matplotlib.pyplot as plt

x = list(range(len(tokens)))
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(x, human_scores, '-',  c='black', label='human', alpha=0.7, linewidth=3)
plt.plot(x, method_scores['nltk'], '--', c='blue', label='nltk', alpha=0.7, linewidth=3)
plt.plot(x, method_scores['nrc'], '-.', c='red', label='nrc', alpha=0.7, linewidth=3)
plt.legend()
plt.title("Sentiment scores")
plt.show()

print(story)

### Discuss 

Do these scores make sense? 

* We can look at specific instances, like NRC on the first sentence (next code block)
* **Is this a happy story or not?**
 * Do the summary scores reflect our judgment about that?
 * If not, why not and how could we improve those scores?

In [ ]:
# Token-level NRC scores for example sentence 1
for word in tokens[0]:
    print(f'{word.ljust(11)}{word_sentiment_score(word)}')

In [ ]:
# Use Seaborn to plot data with lowess (local regression) fit
import seaborn as sns

sns.set_context('talk')
fig, ax = plt.subplots(figsize=(12,8))
sns.regplot(x=x, y=human_scores, lowess=True, color='k', label='human')
sns.regplot(x=x, y=method_scores['nrc'], lowess=True, color='r', label='nrc')
sns.regplot(x=x, y=method_scores['nltk'], lowess=True, color='b', label='nltk')
plt.legend()
plt.show()

## *Madame Bovary* (Flaubert, 1856/1857)

Expected arc: starts happy, ends sad.

Note that we do not lowercase our text (why not?), nor do we remove stopwords and punctuation (again, why not?). We *could* do both of those things, but would want to be sure that there weren't any name collisions in our text (that is, names that have affective associations when used as common words). We might also want to preserve case and punctuation for other tasks in our processing pipeline, even if we don't need them for our sentiment score.

In [ ]:
# Read and tokenize the novel
bovary_file = os.path.join('..', 'data', 'texts', 'F-Flaubert-Madame_Bovary-1857-M.txt')
with open(bovary_file, 'r') as f:
    bovary_text = f.read()
bovary = [word_tokenize(sent) for sent in sent_tokenize(bovary_text)]
print("Sentences:", len(bovary))
print("Total tokens:", sum([len(sent) for sent in bovary]))

Note one small change below: we divide sentence sentiment by sentence length, so that long sentences don't count more than short ones.

In [ ]:
# Score it using NLTK and NRC methods
bovary_scores = {}
for method in methods:
    sentence_scores = []
    for sent in bovary:
        sentence_score = 0
        for word in sent:
            word_score = word_sentiment_score(word, method=method, lex=methods[method])
            if word_score != None:
                sentence_score += word_score
        sentence_scores.append(sentence_score/len(sent))
    bovary_scores[method] = sentence_scores
    print("Method:", method)
    print("Summary score:", round(sum(bovary_scores[method]),2),'\n')

In [ ]:
# Plot results with 4th-order polynomial fit (why?)
fig, ax = plt.subplots(figsize=(12,8))
x_bov = [i/len(bovary) for i in range(len(bovary))] # normalize length as fraction of all sentences
sns.regplot(x=x_bov, y=bovary_scores['nltk'], order=4, scatter=False, color='b', label=None)
sns.regplot(x=x_bov, y=bovary_scores['nltk'], x_bins=20, scatter=True, fit_reg=False, color='b', label='nltk')
sns.regplot(x=x_bov, y=bovary_scores['nrc'], order=4, scatter=False, color='r', label=None)
sns.regplot(x=x_bov, y=bovary_scores['nrc'], x_bins=20, scatter=True, fit_reg=False, color='r', label='nrc')
plt.title("Sentiment in $\it{Madame\ Bovary}$")
plt.xlabel("Narrative time")
plt.ylabel("Average binned sentiment")
plt.legend()
plt.show()

**Discuss:** How would you compare and evaluate the results of these two methods on this text?

## Where to go from here

* More validation!
  * Is your lexicon good? For this kind of text? Written at this time?
* Other dictionaries
  * Develop your own?
  * See Jurafsky and Martin for clever and/or complex ideas
    * Example: tag adjectives. Those that appear on either side of the token `and` probably have the same sentiment polarity; those linked by `but` are likely opposites.
    * Lots of embedding-based approaches, too.
* Other aspects of sentiment/emotion/affect 
  * NRC 'anger', 'surprise', 'trust', etc.
* Other texts and other *types* of text
* Combine sentiment with other kinds of text- and sentence-level scoring
  * Gender, time period (linguistic drift, yikes!), translations of the same text, news coverage of candidates, ...
* More ideas?